In [43]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="inkar10", port=5432)
cur = conn.cursor()
cur.execute("""
    CREATE OR REPLACE FUNCTION firstname_firstletter(letter VARCHAR)
    RETURNS TABLE(
        ID INT,
        FirstName VARCHAR,
        LastName VARCHAR,
        PhoneNumber VARCHAR  
    ) 
    LANGUAGE plpgsql
    AS
    $$
    BEGIN 
        RETURN QUERY
            SELECT
                phonebook.id,
                phonebook.FirstName,
                phonebook.LastName,
                phonebook.PhoneNumber
            FROM 
                phonebook
            WHERE
                phonebook.FirstName LIKE letter || '%';
    END;
    $$;
""")
conn.commit()
cur.execute("SELECT * FROM firstname_firstletter(%s);", ('C',))
results = cur.fetchall()
for row in results:
    print(row)
conn.commit()
cur.close()
conn.close()

(1, 'Cassandra', 'Nunez', '773005940992')
(8, 'Christopher', 'Stevens', '775730792182')


In [44]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="inkar10", port=5432)
cur = conn.cursor()
cur.execute("""
    CREATE OR REPLACE PROCEDURE create_update(
        fname VARCHAR,
        lname VARCHAR,
        number VARCHAR
    )
    LANGUAGE plpgsql
    AS $$
    BEGIN
        UPDATE phonebook
        SET phonenumber = number
        WHERE firstname = fname AND lastname = lname;
        
        IF NOT FOUND THEN
            INSERT INTO phonebook (firstname, lastname, phonenumber)
            VALUES (fname, lname, number);
        END IF;
    END;
    $$;
    """)
conn.commit()
cur.execute("CALL create_update(%s, %s, %s)", ("John", "Doe", "771234156766"))
conn.commit()
cur.execute("""SELECT * FROM Phonebook""")
results = cur.fetchall()
for row in results:
        print(row)
cur.close()
conn.close()

(1, 'Cassandra', 'Nunez', '773005940992')
(2, 'Julie', 'Lynn', '772659178223')
(3, 'Joseph', 'Martin', '771572766394')
(4, 'Ronald', 'Lane', '777014770715')
(5, 'Michelle', 'Cruz', '775544215437')
(6, 'Daniel', 'Perez', '773495615643')
(7, 'Sandra', 'Snyder', '779204487871')
(8, 'Christopher', 'Stevens', '775730792182')
(9, 'Laura', 'Little', '779847641506')
(10, 'Patrick', 'Porter', '779511474188')
(11, 'Ashley', 'Higgins', '778694209761')
(13, 'Megan', 'Fletcher', '770738916607')
(14, 'Brian', 'Stone', '773168847975')
(15, 'Kelly', 'Waters', '776221023384')
(12, 'First', 'Gonzales', '777519449302')
(18, 'Askhat', 'Yergaliyev', '770012345678')
(19, 'Daniyar', 'Tulegenov', '770012345671')
(17, 'John', 'Doe', '771234156766')


In [29]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="inkar10", port=5432)
cur = conn.cursor()
cur.execute("""
CREATE OR REPLACE PROCEDURE insert_many_users(
    first_names TEXT[],
    last_names TEXT[],
    phone_numbers TEXT[],
    INOUT invalid_data TEXT[] DEFAULT NULL
)
LANGUAGE plpgsql
AS $$
DECLARE
    i INTEGER;
BEGIN
    invalid_data := COALESCE(invalid_data, ARRAY[]::TEXT[]);
    
    FOR i IN 1..array_length(first_names, 1) LOOP
        IF phone_numbers[i] ~ '^7\\d{11}$' THEN
            UPDATE phonebook
            SET phonenumber = phone_numbers[i]
            WHERE firstname = first_names[i]
              AND lastname = last_names[i];

            IF NOT FOUND THEN
                INSERT INTO phonebook (firstname, lastname, phonenumber)
                VALUES (first_names[i], last_names[i], phone_numbers[i]);
            END IF;
        ELSE
            invalid_data := array_append(
                invalid_data,
                first_names[i] || ' ' || last_names[i] || ': ' || phone_numbers[i]
            );
        END IF;
    END LOOP;
END;
$$;
""")

cur.execute("""
DO $$
DECLARE
    invalid_entries TEXT[];
BEGIN
    CALL insert_many_users(
        ARRAY['Askhat', 'Daniyar', 'Aliya'],
        ARRAY['Yergaliyev', 'Tulegenov', 'Samatova'],
        ARRAY['770012345678', '770012345671', '7700111212233'],
        invalid_entries
    );
    
    RAISE NOTICE 'Invalid entries: %', invalid_entries;
END;
$$;
""")

cur.execute("SELECT * FROM phonebook")
for row in cur.fetchall():
    print(row)

conn.commit()
cur.close()
conn.close()

(1, 'Cassandra', 'Nunez', '773005940992')
(2, 'Julie', 'Lynn', '772659178223')
(3, 'Joseph', 'Martin', '771572766394')
(4, 'Ronald', 'Lane', '777014770715')
(5, 'Michelle', 'Cruz', '775544215437')
(6, 'Daniel', 'Perez', '773495615643')
(7, 'Sandra', 'Snyder', '779204487871')
(8, 'Christopher', 'Stevens', '775730792182')
(9, 'Laura', 'Little', '779847641506')
(10, 'Patrick', 'Porter', '779511474188')
(11, 'Ashley', 'Higgins', '778694209761')
(13, 'Megan', 'Fletcher', '770738916607')
(14, 'Brian', 'Stone', '773168847975')
(15, 'Kelly', 'Waters', '776221023384')
(12, 'First', 'Gonzales', '777519449302')
(16, 'Inkar', 'Askarbekova', '775000123654')
(17, 'John', 'Doe', '771234156766')
(18, 'Askhat', 'Yergaliyev', '770012345678')
(19, 'Daniyar', 'Tulegenov', '770012345671')


In [38]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="inkar10", port=5432)
cur = conn.cursor()
cur.execute("""
CREATE OR REPLACE FUNCTION pagination(
    search_term TEXT DEFAULT NULL,
    limit_val INTEGER DEFAULT 10,
    offset_val INTEGER DEFAULT 0
)
RETURNS TABLE(
    id INTEGER,
    firstname VARCHAR(50),
    lastname VARCHAR(50),
    phonenumber VARCHAR(12)
)
LANGUAGE plpgsql
AS $$
BEGIN
    RETURN QUERY
    SELECT phonebook.id, phonebook.firstname, phonebook.lastname, phonebook.phonenumber
    FROM phonebook
    WHERE 
        search_term IS NULL OR
        phonebook.firstname ILIKE '%' || search_term || '%' OR
        phonebook.lastname ILIKE '%' || search_term || '%' OR
        phonebook.phonenumber ILIKE '%' || search_term || '%'
    ORDER BY phonebook.lastname, phonebook.firstname
    LIMIT limit_val
    OFFSET offset_val;
END;
$$ 
""")
conn.commit()
cur.execute("""SELECT * FROM pagination()""")
for row in cur.fetchall():
    print(row)
conn.commit()
cur.close()
conn.close()

(16, 'Inkar', 'Askarbekova', '775000123654')
(5, 'Michelle', 'Cruz', '775544215437')
(17, 'John', 'Doe', '771234156766')
(13, 'Megan', 'Fletcher', '770738916607')
(12, 'First', 'Gonzales', '777519449302')
(11, 'Ashley', 'Higgins', '778694209761')
(4, 'Ronald', 'Lane', '777014770715')
(9, 'Laura', 'Little', '779847641506')
(2, 'Julie', 'Lynn', '772659178223')
(3, 'Joseph', 'Martin', '771572766394')


In [42]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="inkar10", port=5432)
cur = conn.cursor()
cur.execute("""
CREATE OR REPLACE PROCEDURE delete_user(
    IN fname VARCHAR,
    IN lname VARCHAR,
    IN phone VARCHAR
)
LANGUAGE plpgsql
AS $$
BEGIN
    IF fname IS NOT NULL AND lname IS NOT NULL THEN
        DELETE FROM phonebook
        WHERE firstname = fname AND lastname = lname;
    ELSIF phone IS NOT NULL THEN
        DELETE FROM phonebook
        WHERE phonenumber = phone;
    END IF;
END;
$$;
""")
conn.commit()
cur.execute("""
    CALL delete_user(%s, %s, %s);
""", ('Inkar', 'Askarbekova', None))
cur.execute("""SELECT * FROM phonebook""")
for row in cur.fetchall():
    print(row)
conn.commit()
cur.close()
conn.close()

(1, 'Cassandra', 'Nunez', '773005940992')
(2, 'Julie', 'Lynn', '772659178223')
(3, 'Joseph', 'Martin', '771572766394')
(4, 'Ronald', 'Lane', '777014770715')
(5, 'Michelle', 'Cruz', '775544215437')
(6, 'Daniel', 'Perez', '773495615643')
(7, 'Sandra', 'Snyder', '779204487871')
(8, 'Christopher', 'Stevens', '775730792182')
(9, 'Laura', 'Little', '779847641506')
(10, 'Patrick', 'Porter', '779511474188')
(11, 'Ashley', 'Higgins', '778694209761')
(13, 'Megan', 'Fletcher', '770738916607')
(14, 'Brian', 'Stone', '773168847975')
(15, 'Kelly', 'Waters', '776221023384')
(12, 'First', 'Gonzales', '777519449302')
(17, 'John', 'Doe', '771234156766')
(18, 'Askhat', 'Yergaliyev', '770012345678')
(19, 'Daniyar', 'Tulegenov', '770012345671')
